In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/project/section-2/mlruns/1', creation_time=1719853039311, experiment_id='1', last_update_time=1719853039311, lifecycle_stage='active', name='project-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1719852963603, experiment_id='0', last_update_time=1719852963603, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.accuracy < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.accuracy ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['accuracy']:.4f}")

run id: efef04d3fc3a4bee94cd9c03648f82e2, rmse: 0.9393
run id: 60f0b5e5336f474c8e034e32e9275f1e, rmse: 0.9393


In [ ]:
efef04d3fc3a4bee94cd9c03648f82e2

Interacting with Model Registry

In [11]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
run_id = "bfb2434a50b54db4b1297dc168dbb7a0"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1719569546783, current_stage='None', description=None, last_updated_timestamp=1719569546783, name='nyc-taxi-regressor', run_id='bfb2434a50b54db4b1297dc168dbb7a0', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/bfb2434a50b54db4b1297dc168dbb7a0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [14]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1719567707948, description='', last_updated_timestamp=1719569546783, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1719569546783, current_stage='None', description=None, last_updated_timestamp=1719569546783, name='nyc-taxi-regressor', run_id='bfb2434a50b54db4b1297dc168dbb7a0', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/bfb2434a50b54db4b1297dc168dbb7a0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>], name='nyc-taxi-regressor', tags={'model': 'xgboost'}>]

Verify the models registered wrt its stages

In [15]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


/tmp/ipykernel_49609/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


Manually transition the stage of particular version of model

In [16]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_49609/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1719569546783, current_stage='Staging', description=None, last_updated_timestamp=1719569835831, name='nyc-taxi-regressor', run_id='bfb2434a50b54db4b1297dc168dbb7a0', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/bfb2434a50b54db4b1297dc168dbb7a0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

Update the registered models with its transition details wrt date and time

It is like a description

In [17]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1719569546783, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-06-28', last_updated_timestamp=1719570036197, name='nyc-taxi-regressor', run_id='bfb2434a50b54db4b1297dc168dbb7a0', run_link=None, source='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1/bfb2434a50b54db4b1297dc168dbb7a0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

Check the staged version code rmse is predicted good with the test data and move to production

we can move to prod and also can check the predictions

In [31]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [32]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [ ]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [27]:
import pickle

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df, dv)

In [ ]:
target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)